In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats

##### 1. One Sample K-S Test:

In [ ]:
# Setting the seed for code reproducibility
np.random.seed(123)

#generate dataset of 100 values that follow a Poisson distribution with mean=5
data = pd.DataFrame(np.random.poisson(lam=5,size=100), columns=['arrival_time'])
print(data.shape)
data.head()

In [ ]:
# Perform Kolmogorov-Smirnov test to check if the arrival_time is from a normal distribution

test_obj = stats.kstest(data, 'norm',N=100)
test_obj

In [ ]:
np.round(test_obj.pvalue,3)

##### 2. Two Sample K-S Test:

In [ ]:
# Setting the seed for code reproducibility
np.random.seed(123)

#generate dataset of 100 values that follow a Poisson distribution
data1 = pd.DataFrame(np.random.poisson(lam=5,size=100), columns=['arrival_time'])
# data2 = pd.DataFrame(np.random.normal(7,2,100), columns=['arrival_time'])
data2 = pd.DataFrame(np.random.poisson(lam=5,size=100), columns=['arrival_time'])
print(data1.shape)
display(data1.head())
print(data2.shape)
display(data2.head())

In [ ]:
# Perform Kolmogorov-Smirnov test to check if the arrival_time from two experiments is same or not

test_obj = stats.ks_2samp(data1.arrival_time, data2.arrival_time)
test_obj

In [ ]:
np.round(test_obj.pvalue,4)

In [ ]:
import numpy as np

def calculate_psi(expected, actual, buckettype='bins', buckets=10, axis=0):
    '''Calculate the PSI (population stability index) across all variables
    Args:
       expected: numpy matrix of original values
       actual: numpy matrix of new values, same size as expected
       buckettype: type of strategy for creating buckets, bins splits into even splits, quantiles splits into quantile buckets
       buckets: number of quantiles to use in bucketing variables
       axis: axis by which variables are defined, 0 for vertical, 1 for horizontal
    Returns:
       psi_values: ndarray of psi values for each variable
    Author:
       Matthew Burke
       github.com/mwburke
       worksofchart.com
    '''

    def psi(expected_array, actual_array, buckets):
        '''Calculate the PSI for a single variable
        Args:
           expected_array: numpy array of original values
           actual_array: numpy array of new values, same size as expected
           buckets: number of percentile ranges to bucket the values into
        Returns:
           psi_value: calculated PSI value
        '''

        def scale_range (input, min, max):
            input += -(np.min(input))
            input /= np.max(input) / (max - min)
            input += min
            return input


        breakpoints = np.arange(0, buckets + 1) / (buckets) * 100

        if buckettype == 'bins':
            breakpoints = scale_range(breakpoints, np.min(expected_array), np.max(expected_array))
        elif buckettype == 'quantiles':
            breakpoints = np.stack([np.percentile(expected_array, b) for b in breakpoints])



        expected_percents = np.histogram(expected_array, breakpoints)[0] / len(expected_array)
        actual_percents = np.histogram(actual_array, breakpoints)[0] / len(actual_array)

        def sub_psi(e_perc, a_perc):
            '''Calculate the actual PSI value from comparing the values.
               Update the actual value to a very small number if equal to zero
            '''
            if a_perc == 0:
                a_perc = 0.0001
            if e_perc == 0:
                e_perc = 0.0001

            value = (e_perc - a_perc) * np.log(e_perc / a_perc)
            return(value)

        psi_value = np.sum(sub_psi(expected_percents[i], actual_percents[i]) for i in range(0, len(expected_percents)))

        return(psi_value)

    if len(expected.shape) == 1:
        psi_values = np.empty(len(expected.shape))
    else:
        psi_values = np.empty(expected.shape[axis])

    for i in range(0, len(psi_values)):
        if len(psi_values) == 1:
            psi_values = psi(expected, actual, buckets)
        elif axis == 0:
            psi_values[i] = psi(expected[:,i], actual[:,i], buckets)
        elif axis == 1:
            psi_values[i] = psi(expected[i,:], actual[i,:], buckets)

    return(psi_values)

In [ ]:
# Setting the seed for code reproducibility
np.random.seed(123)

#generate dataset of 100 values that follow a Poisson distribution with mean=5
data1 = pd.DataFrame(np.random.poisson(lam=5,size=100), columns=['arrival_time'])
# data1 = pd.DataFrame(np.random.normal(7,2,100), columns=['arrival_time'])
data2 = pd.DataFrame(np.random.normal(7,2,100), columns=['arrival_time'])
print(data1.shape)
display(data1.head())
print(data2.shape)
display(data2.head())

In [ ]:
psi_val = calculate_psi(data1.arrival_time,data2.arrival_time)
print(psi_val)

In [ ]:
# example of calculating the kl divergence between two mass functions
from math import log2

# calculate the kl divergence
def kl_divergence(p, q):
	return sum(p[i] * log2(p[i]/q[i]) for i in range(len(p)))

# define distributions
p = [0.10, 0.40, 0.50]
q = [0.80, 0.15, 0.05]
# calculate (P || Q)
kl_pq = kl_divergence(p, q)
print('KL(P || Q): %.3f bits' % kl_pq)
# calculate (Q || P)
kl_qp = kl_divergence(q, p)
print('KL(Q || P): %.3f bits' % kl_qp)

In [ ]:
# example of calculating the kl divergence (relative entropy) with scipy
from scipy.special import rel_entr
# define distributions
p = [0.10, 0.40, 0.50]
q = [0.80, 0.15, 0.05]
# calculate (P || Q)
kl_pq = rel_entr(p, q)
print('KL(P || Q): %.3f nats' % sum(kl_pq))
# calculate (Q || P)
kl_qp = rel_entr(q, p)
print('KL(Q || P): %.3f nats' % sum(kl_qp))

In [ ]:
# example of calculating the js divergence between two mass functions
from math import log2
from math import sqrt
from numpy import asarray

# calculate the kl divergence
def kl_divergence(p, q):
	return sum(p[i] * log2(p[i]/q[i]) for i in range(len(p)))

# calculate the js divergence
def js_divergence(p, q):
	m = 0.5 * (p + q)
	return 0.5 * kl_divergence(p, m) + 0.5 * kl_divergence(q, m)

# define distributions
p = asarray([0.10, 0.40, 0.50])
q = asarray([0.80, 0.15, 0.05])
# calculate JS(P || Q)
js_pq = js_divergence(p, q)
print('JS(P || Q) divergence: %.3f bits' % js_pq)
print('JS(P || Q) distance: %.3f' % sqrt(js_pq))
# calculate JS(Q || P)
js_qp = js_divergence(q, p)
print('JS(Q || P) divergence: %.3f bits' % js_qp)
print('JS(Q || P) distance: %.3f' % sqrt(js_qp))

In [ ]:
# calculate the jensen-shannon distance metric
from scipy.spatial.distance import jensenshannon
from numpy import asarray
# define distributions
p = asarray([0.10, 0.40, 0.50])
q = asarray([0.80, 0.15, 0.05])
# calculate JS(P || Q)
js_pq = jensenshannon(p, q, base=2)
print('JS(P || Q) Distance: %.3f' % js_pq)
# calculate JS(Q || P)
js_qp = jensenshannon(q, p, base=2)
print('JS(Q || P) Distance: %.3f' % js_qp)

In [ ]:
# Setting the seed for code reproducibility
np.random.seed(123)

#generate dataset of 100 values that follow a Poisson distribution with mean=5
data1 = pd.DataFrame(np.random.poisson(lam=5,size=100), columns=['arrival_time'])
data2 = pd.DataFrame(np.random.normal(7,2,100), columns=['arrival_time'])
print(data1.shape)
display(data1.head())
print(data2.shape)
display(data2.head())

In [ ]:
# Compute the first Wasserstein distance between two 1D distributions.

wd = stats.wasserstein_distance(data1.arrival_time,data2.arrival_time)
print("For different distributions: "+str(wd))

wd = stats.wasserstein_distance(data1.arrival_time,data1.arrival_time)
print("For same distributions: "+str(wd))

In [ ]:
# creating reference and target dataframe with categorical column
d1 = ['A','A','A','A','B','B','C','C','C']
d2 = ['A','A','B','B','B','B','B','C','C']
data_ref = pd.DataFrame(d1, columns=['ticket_category_ref'])
print(data_ref.shape)
display(data_ref)

data_target = pd.DataFrame(d2, columns=['ticket_category_tar'])
print(data_target.shape)
display(data_target)

In [ ]:
cross_tab = pd.crosstab(data_ref.ticket_category_ref,data_target.ticket_category_tar)
cross_tab

In [ ]:
# Chi-Square test of independence
# if p < 0.05 then the two categories are not independnt of each other
chi2_stat, pvalue, df, expected_freq = stats.chi2_contingency(cross_tab)
print(pvalue)

In [ ]:
# Chi-Square goodness of fit test
# Make sure you have the frequency of each categories in the same order in both observed and expected
# if p < 0.05 then the two frequencies are not from same distribution
expected = data_target.groupby('ticket_category_tar')['ticket_category_tar'].count().values
observed = data_ref.groupby('ticket_category_ref')['ticket_category_ref'].count().values 

chi2_stat, pvalue = stats.chisquare(f_obs=observed,f_exp=expected)
print(pvalue)

In [ ]:
data_target.groupby('ticket_category_tar')['ticket_category_tar'].count().values

In [ ]:
data_ref.groupby('ticket_category_ref')['ticket_category_ref'].count().values 